In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
# 定义参数
region = 'tianjin'
year = '2016'
startDate = year + '-01-01'
endDate = year + '-12-31'

In [4]:
# # 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# # 天津市
roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# # 武汉市
# roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# # 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
# roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [5]:
# 移除无效像元
# .reproject('EPSG:3857',None,30)
# elevation = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
elevation = ee.Image("users/311605001111/hillshade_" + region)
def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True)).updateMask(outlier)
def maskSR_reverse(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    image_cloud = img.updateMask(mask.remap([0,1],[1,0]))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image_shadow = img.updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True).remap([0,1],[1,0]))
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    image_outlier = img.updateMask(bands.gt(0).rename('outlier'))
    return ee.ImageCollection([image_cloud,image_shadow,image_outlier]).sum()


# 图像可视化参数
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}

## 指数的计算
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('(B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7)/10000',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi).addBands(ndvi_mndwi)

# 计算研究区域内，影像面积
def ImageArea(image):
    image_area = image.select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    imageareas = image_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': roi,
        'scale': 1000,
        'maxPixels': 1e14,
        'bestEffort': True,
    })    
    return image.set({'imagearea': imageareas.get('B2')})


In [6]:
# l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
#        .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
#        .filterBounds(roi) \
#        .filterDate(startDate, endDate)
           
# l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
#        .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
#        .filterBounds(roi) \
#        .filterDate(startDate, endDate)          
# landsat_images = l8.merge(l7).map(water_index).map(maskSR)
# Map.addLayer(l8.map(maskSR).mosaic().clip(roi), visParams,'image mosaic')
# print(l7.size().getInfo())
# print(l8.size().getInfo())
# print(landsat_images.size().getInfo())

In [7]:
l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
       .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate(startDate, endDate)  
l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
       .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate(startDate, endDate)           
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
       .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate(startDate, endDate)
landsat_images = l8.merge(l7).merge(l5).map(water_index)
Map.addLayer(l8.map(maskSR).mosaic().clip(roi), visParams,'image mosaic')
print(l5.size().getInfo( ))
print(l7.size().getInfo())
print(l8.size().getInfo())
print(landsat_images.size().getInfo())

0
102
123
225


# 筛选得出采样影像

## 方案一

In [ ]:
# 筛选图像
image_area = landsat_images.mosaic().select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
areas = image_area.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': roi,
    'scale': 1000,
    'maxPixels': 1e14
})
print(areas.getInfo())
# 筛选影像，影像覆盖研究区的范围 > 研究区的1/3
sample_image = landsat_images.map(ImageArea).filter(ee.Filter.gt('imagearea',ee.Number(areas.get('B2')).divide(3)))
dates = sample_image.aggregate_array('system:time_start').sort().map(lambda d: ee.Date(d).format('YYYY-MM-dd'))
print(sample_image.size().getInfo())
# print(sample_image.aggregate_array('system:id').getInfo())
print(dates.getInfo())

landsat_image = sample_image.map(lambda i : i.setMulti({'path_row': ee.String(i.get('system:id')).slice(-15,-9),'month':ee.Number.parse(ee.String(i.get('system:index')).slice(-4,-2))}))
path_row = landsat_image.distinct('path_row').aggregate_array('path_row')
print(landsat_image.aggregate_array('month').getInfo())
print(path_row.getInfo())

## 方案二

In [ ]:
# 影像筛选函数
def filter_image(id):
    # l8
    images_l8 = l8.map(lambda i:i.setMulti({'path_row': ee.String(i.get('system:id')).slice(-15,-9),'month':ee.Number.parse(ee.String(i.get('system:id')).slice(-4,-2))}))
    month_l8 = images_l8.map(lambda i:i.set({'img_id':i.get('system:index')})).filter(ee.Filter.eq('path_row',id))
    first_quarter_l8 = ee.ImageCollection(month_l8.filter(ee.Filter.lte('month',3)).filter(ee.Filter.gte('month',1)).randomColumn('random',1,'uniform').sort('random')).first()
    second_quarter_l8 = ee.ImageCollection(month_l8.filter(ee.Filter.lte('month',6)).filter(ee.Filter.gte('month',4)).randomColumn('random',1,'uniform').sort('random')).first()
    third_quarter_l8 = ee.ImageCollection(month_l8.filter(ee.Filter.lte('month',9)).filter(ee.Filter.gte('month',7)).randomColumn('random',1,'uniform').sort('random')).first()
    fourth_quarter_l8 = ee.ImageCollection(month_l8.filter(ee.Filter.lte('month',12)).filter(ee.Filter.gte('month',10)).randomColumn('random',1,'uniform').sort('random')).first()
    # l7
    images_l7 = l7.map(lambda i:i.setMulti({'path_row': ee.String(i.get('system:id')).slice(-15,-9),'month':ee.Number.parse(ee.String(i.get('system:id')).slice(-4,-2))}))
    month_l7 = images_l7.map(lambda i:i.set({'img_id':i.get('system:index')})).filter(ee.Filter.eq('path_row',id))
    first_quarter_l7 = ee.ImageCollection(month_l7.filter(ee.Filter.lte('month',3)).filter(ee.Filter.gte('month',1)).randomColumn('random',1,'uniform').sort('random')).first()
    second_quarter_l7 = ee.ImageCollection(month_l7.filter(ee.Filter.lte('month',6)).filter(ee.Filter.gte('month',4)).randomColumn('random',1,'uniform').sort('random')).first()
    third_quarter_l7 = ee.ImageCollection(month_l7.filter(ee.Filter.lte('month',9)).filter(ee.Filter.gte('month',7)).randomColumn('random',1,'uniform').sort('random')).first()
    fourth_quarter_l7 = ee.ImageCollection(month_l7.filter(ee.Filter.lte('month',12)).filter(ee.Filter.gte('month',10)).randomColumn('random',1,'uniform').sort('random')).first()
    # l5
    images_l5 = l5.map(lambda i:i.setMulti({'path_row': ee.String(i.get('system:id')).slice(-15,-9),'month':ee.Number.parse(ee.String(i.get('system:id')).slice(-4,-2))}))
    month_l5 = images_l5.map(lambda i:i.set({'img_id':i.get('system:index')})).filter(ee.Filter.eq('path_row',id))
    first_quarter_l5 = ee.ImageCollection(month_l5.filter(ee.Filter.lte('month',3)).filter(ee.Filter.gte('month',1)).randomColumn('random',1,'uniform').sort('random')).first()
    second_quarter_l5 = ee.ImageCollection(month_l5.filter(ee.Filter.lte('month',6)).filter(ee.Filter.gte('month',4)).randomColumn('random',1,'uniform').sort('random')).first()
    third_quarter_l5 = ee.ImageCollection(month_l5.filter(ee.Filter.lte('month',9)).filter(ee.Filter.gte('month',7)).randomColumn('random',1,'uniform').sort('random')).first()
    fourth_quarter_l5 = ee.ImageCollection(month_l5.filter(ee.Filter.lte('month',12)).filter(ee.Filter.gte('month',10)).randomColumn('random',1,'uniform').sort('random')).first()
    img_col = ee.ImageCollection([first_quarter_l8,second_quarter_l8,third_quarter_l8,fourth_quarter_l8,first_quarter_l7,second_quarter_l7,third_quarter_l7,fourth_quarter_l7,first_quarter_l5,second_quarter_l5,third_quarter_l5,fourth_quarter_l5])
    return img_col

## 计算唯一的行列号
l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate)  
l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate)           
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate)
region_images = l8.merge(l7).merge(l5)
landsat_image = region_images.map(lambda i : i.setMulti({'path_row': ee.String(i.get('system:id')).slice(-15,-9),'month':ee.Number.parse(ee.String(i.get('system:id')).slice(-4,-2))}))
path_row = landsat_image.distinct('path_row').aggregate_array('path_row')
print(path_row.getInfo())

## 计算在每个行列号上选择的月份和卫星类型
select_image = path_row.map(filter_image)
number = path_row.size().getInfo()
print(path_row.size().getInfo())
for i in range(0,number):
    print(ee.ImageCollection(select_image .get(i)).aggregate_array('month').getInfo())
    print(ee.ImageCollection(select_image .get(i)).aggregate_array('SATELLITE').getInfo())

## 合并在各个行列号上选择的影像
img_col = ee.ImageCollection([])
for i in range(0,number):
    img_col = img_col.merge(ee.ImageCollection(select_image.get(i)))
sample_image = img_col.map(water_index).map(maskSR)
print(sample_image.size().getInfo())
print(sample_image.aggregate_array('path_row').getInfo())
print(sample_image.aggregate_array('month').getInfo())

## 方案三

In [ ]:
## 计算无效像元比
def filter_image(img):
    validPixel = maskSR(img).select('pixel_qa').gt(0).rename('waterclass').float()
    invalidPixel = maskSR_reverse(img).select('pixel_qa').gt(0).remap([0,1],[1,0]).rename('waterclass').float()
    class_image = ee.ImageCollection([invalidPixel,validPixel]).sum()
    invalidPixels = class_image.eq(0).multiply(ee.Image.pixelArea()).divide(1e6)
    invalidarea = invalidPixels.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 200,'maxPixels': 1e14,'tileScale': 2}).get('waterclass')
    region = class_image.gte(0).multiply(ee.Image.pixelArea()).divide(1e6)
    regionarea = region.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 200,'maxPixels': 1e14,'tileScale': 2}).get('waterclass')
    rate = ee.Number(invalidarea).divide(ee.Number(regionarea)).multiply(100)
    return img.set({'invalid_rate':rate})

sample_image = landsat_images.map(filter_image).filter(ee.Filter.lt('invalid_rate',85)).map(maskSR)
print(sample_image.size().getInfo())
# print(sample_image.aggregate_array('invalid_rate').getInfo())

In [ ]:
# sample_image = l8.map(water_index).map(filter_image).filter(ee.Filter.lt('invalid_rate',85)).map(maskSR)
# print(sample_image.size().getInfo())
# print(sample_image.aggregate_array('invalid_rate').getInfo())

# sample_image = l7.map(water_index).map(filter_image).filter(ee.Filter.lt('invalid_rate',85)).map(maskSR)
# print(sample_image.size().getInfo())
# print(sample_image.aggregate_array('invalid_rate').getInfo())

## 方案四

In [8]:
sample_image = landsat_images.map(maskSR)
print(sample_image.size().getInfo())

225


# 多水体频率数据集的类别范围确定

## 方案一

In [ ]:
#### 设置年份
JRC_id = 'JRC/GSW1_3/YearlyHistory/' + year
Maryland_id = 'users/311605001111/Maryland/Maryland_nationwide_' + year
# ndvi_mndwi水体频率
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
water_layer = landsat_images.map(lambda i : i.select('ndvi_mndwi').lt(0)).sum()
validPixel_layer = landsat_images.count().select('B3').clip(roi).rename('count')
index_waterfrequency = water_layer.select('ndvi_mndwi').divide(validPixel_layer.select('count')).select('ndvi_mndwi').rename('frequency')
index_permanent = index_waterfrequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
index_season= ee.ImageCollection([index_waterfrequency.gte(0.25),index_waterfrequency.lt(0.75)]).sum().eq(2).rename('waterclass')
index_waterclass = ee.ImageCollection([index_permanent,index_season,basemap]).sum()
# JRC
JRC = ee.Image(JRC_id).clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
JRC_waterclass = ee.ImageCollection([JRC,basemap]).sum()
# Maryland
Maryland = ee.Image(Maryland_id).clip(roi).select('b1').rename('waterclass')
Maryland_permanent = Maryland.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
Maryland_season= ee.ImageCollection([Maryland.gte(25),Maryland.lt(75)]).sum().eq(2)
Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()
# 三者的交集
permanentWaterExtent = ee.ImageCollection([index_waterclass.eq(2),JRC_waterclass.eq(2),Maryland_waterclass.eq(2)]).sum().eq(3)
landExtent = ee.ImageCollection([index_waterclass.eq(0),JRC_waterclass.eq(0),Maryland_waterclass.eq(0)]).sum().eq(3)
seasonWaterExtent = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')

# Map.addLayer(permanentWaterExtent.selfMask(),{'palette':['blue']},"permanent_water")
# Map.addLayer(seasonWaterExtent.selfMask(),{'palette':['green']},"season water")

## 方案二

In [ ]:
#### 设置年份
JRC_id = 'JRC/GSW1_3/YearlyHistory/' + year
Maryland_id = 'users/311605001111/Maryland/Maryland_nationwide_' + year
# ndvi_mndwi水体频率
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
water_layer = landsat_images.map(lambda i : i.select('ndvi_mndwi').lt(0)).sum()
validPixel_layer = landsat_images.count().select('B3').clip(roi).rename('count')
index_waterfrequency = water_layer.select('ndvi_mndwi').divide(validPixel_layer.select('count')).select('ndvi_mndwi').rename('frequency')
index_permanent = index_waterfrequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
index_season= ee.ImageCollection([index_waterfrequency.gte(0.25),index_waterfrequency.lt(0.75)]).sum().eq(2).rename('waterclass')
index_waterclass = ee.ImageCollection([index_permanent,index_season,basemap]).sum()
# JRC
JRC = ee.Image(JRC_id).clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
JRC_waterclass = ee.ImageCollection([JRC,basemap]).sum()
# Maryland
Maryland = ee.Image(Maryland_id).clip(roi).select('b1').rename('waterclass')
Maryland_permanent = Maryland.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
Maryland_season= ee.ImageCollection([Maryland.gte(25),Maryland.lt(75)]).sum().eq(2)
Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()
# 三者的交集
permanentWaterExtent = ee.ImageCollection([index_waterclass.eq(2),JRC_waterclass.eq(2),Maryland_waterclass.eq(2)]).sum().eq(3)
landExtent = ee.ImageCollection([index_waterclass.eq(0),JRC_waterclass.eq(0),Maryland_waterclass.eq(0)]).sum().eq(3)
seasonWaterExtent = ee.ImageCollection([index_waterclass.eq(1),JRC_waterclass.eq(1),Maryland_waterclass.eq(1)]).sum().eq(3)

##  方案三

In [ ]:
#### 设置年份
JRC_id = 'JRC/GSW1_3/YearlyHistory/' + year
Maryland_id = 'users/311605001111/Maryland/Maryland_nationwide_' + year
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
# JRC
JRC = ee.Image(JRC_id).clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
JRC_waterclass = ee.ImageCollection([JRC,basemap]).sum()
# Maryland
Maryland = ee.Image(Maryland_id).clip(roi).select('b1').rename('waterclass')
Maryland_permanent = Maryland.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
Maryland_season= ee.ImageCollection([Maryland.gte(25),Maryland.lt(75)]).sum().eq(2)
Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()
# 三者的交集
permanentWaterExtent = ee.ImageCollection([JRC_waterclass.eq(2),Maryland_waterclass.eq(2)]).sum().eq(2)
landExtent = ee.ImageCollection([JRC_waterclass.eq(0),Maryland_waterclass.eq(0)]).sum().eq(2)
seasonWaterExtent = ee.ImageCollection([JRC_waterclass.eq(1),Maryland_waterclass.eq(1)]).sum().eq(2)

## 方案四

In [ ]:
#### 设置年份
Maryland_id = 'users/311605001111/Maryland/Maryland_nationwide_' + year
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
# JRC
dataset = ee.ImageCollection('JRC/GSW1_3/MonthlyHistory').filter(ee.Filter.eq('year',int(year)))
waterpixel = dataset.map(lambda i : i.clip(roi).eq(2)).sum()
validpixel = dataset.map(lambda i : i.clip(roi).gt(0)).sum()
waterfrequency = waterpixel.divide(validpixel).multiply(100).rename('waterclass')
JRC_permanent = waterfrequency.select('waterclass').gt(75).remap([0,1],[0,2]).rename('waterclass')
JRC_season= ee.ImageCollection([waterfrequency.gte(25),waterfrequency.lte(75)]).sum().eq(2)
JRC_waterclass = ee.ImageCollection([JRC_permanent,JRC_season,basemap]).sum()
# Maryland
Maryland = ee.Image(Maryland_id).clip(roi).select('b1').rename('waterclass')
Maryland_permanent = Maryland.select('waterclass').gt(75).remap([0,1],[0,2]).rename('waterclass')
Maryland_season= ee.ImageCollection([Maryland.gte(25),Maryland.lte(75)]).sum().eq(2)
Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()
# 三者的交集
permanentWaterExtent = ee.ImageCollection([JRC_waterclass.eq(2),Maryland_waterclass.eq(2)]).sum().eq(2)
landExtent = ee.ImageCollection([JRC_waterclass.eq(0),Maryland_waterclass.eq(0)]).sum().eq(2)
seasonWaterExtent_min = ee.ImageCollection([JRC_waterclass.eq(1),Maryland_waterclass.eq(1)]).sum().eq(2)
seasonWaterExtent_max = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')

## 方案五

In [9]:
#### 设置年份
JRC_id = 'JRC/GSW1_3/YearlyHistory/' + year
Maryland_id = 'users/311605001111/MARYLAND/Maryland_nationwide_' + year
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
# JRC
JRC = ee.Image(JRC_id).clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
JRC_waterclass = ee.ImageCollection([JRC,basemap]).sum()
# Maryland
Maryland = ee.Image(Maryland_id).clip(roi).select('b1').rename('waterclass')
Maryland_permanent = Maryland.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
Maryland_season= ee.ImageCollection([Maryland.gte(25),Maryland.lt(75)]).sum().eq(2)
Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()
# 三者的交集
permanentWaterExtent = ee.ImageCollection([JRC_waterclass.eq(2),Maryland_waterclass.eq(2)]).sum().eq(2)
landExtent = ee.ImageCollection([JRC_waterclass.eq(0),Maryland_waterclass.eq(0)]).sum().eq(2)
seasonWaterExtent_min = ee.ImageCollection([JRC_waterclass.eq(1),Maryland_waterclass.eq(1)]).sum().eq(2)
seasonWaterExtent_max = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')

# 采样函数

## 三类，AWEI为0

In [ ]:
### 一般阈值：AWEI=0
# 设置采样个数
number_land = 250
number_water = 500
# 基于图像的样本采集函数
def imageSample(image):
    # 永久性水体
    permanent_points = image.updateMask(permanentWaterExtent).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': number_water,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    water_points = permanent_points.map(lambda i : i.setMulti({'waterclass':1,'Image_id':image.get('system:id')}))
    # 陆地
    land_points = image.updateMask(landExtent).sample(**{
        'region': image.geometry(),
        'scale': 30,
        'numPixels': number_land,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    nowater_points = land_points.map(lambda i : i.setMulti({'waterclass':0,'Image_id':image.get('system:id')}))
    # 季节性水体
    awei = image.updateMask(seasonWaterExtent).select('AWEI').gt(0)
    seasonWater = image.updateMask(awei).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': number_water,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    seasonWater_points = seasonWater.map(lambda i : i.setMulti({'waterclass':1,'Image_id':image.get('system:id')}))
    seasonLand= image.updateMask(awei.remap([0,1],[1,0])).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': number_land,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    seasonLand_points = seasonLand.map(lambda i : i.setMulti({'waterclass':0,'Image_id':image.get('system:id')}))
    sample_points = ee.FeatureCollection([water_points,nowater_points,seasonWater_points,seasonLand_points]).flatten()
    return sample_points

## 只有永久和非水体

In [10]:
# 设置采样个数
# number_land = 150
# number_water = 2000
number_land = 250
number_water = 1500

# 基于图像的样本采集函数
def imageSample(image):
    # 永久性水体
    permanent_points = image.updateMask(permanentWaterExtent).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': number_water,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    water_points = permanent_points.map(lambda i : i.setMulti({'waterclass':1,'point_type':'permanent_water','Image_id':image.get('system:id')}))
    # 陆地
    land_points = image.updateMask(landExtent).sample(**{
        'region': image.geometry(),
        'scale': 30,
        'numPixels': number_land,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    nowater_points = land_points.map(lambda i : i.setMulti({'waterclass':0,'point_type':'no_water','Image_id':image.get('system:id')}))
    sample_points = ee.FeatureCollection([water_points,nowater_points]).flatten()
    return sample_points

## 季节性，先awei,后RF

In [ ]:
training = 2000
collect = 500
def season(image):
    awei = image.updateMask(seasonWaterExtent_min).select('AWEI').gt(0)
    seasonWater = image.updateMask(awei).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': training,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    seasonWater_points = seasonWater.map(lambda i : i.set({'waterclass':1}))
    seasonLand= image.updateMask(awei.remap([0,1],[1,0])).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': training,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    seasonLand_points = seasonLand.map(lambda i : i.set({'waterclass':0}))
    sample_points = ee.FeatureCollection([seasonWater_points,seasonLand_points]).flatten()
    label = 'waterclass'
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
    trainedClassifier = ee.Classifier.smileRandomForest(20).train(sample_points,label,bands)
    classify_image = image.updateMask(seasonWaterExtent_max).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')   
    seasonWater = image.updateMask(classify_image).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': collect,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    sw = seasonWater.map(lambda i : i.setMulti({'waterclass':1,'Image_id':image.get('system:id')}))
    seasonLand = image.updateMask(classify_image.remap([0,1],[1,0])).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': collect,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    sl = seasonLand.map(lambda i : i.setMulti({'waterclass':0,'Image_id':image.get('system:id')}))
    sample_points = ee.FeatureCollection([sw,sl]).flatten()
    return sample_points

## 季节性，先k-mean，后RF

### 先筛选可能只有一类样本的情况

In [ ]:
def filter_image(image):
    samples = image.updateMask(seasonWaterExtent_min).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': training,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    label = 'waterclass'
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
    cluster_samples = ee.FeatureCollection('users/311605001111/daqing_new_noseason2020')
    clusterer = ee.Clusterer.wekaXMeans(2,2).train(cluster_samples,bands)
    q = ee.FeatureCollection(samples.cluster(clusterer))
    q1 = q.filter(ee.Filter.eq('cluster',1)).size()
    q0 = q.filter(ee.Filter.eq('cluster',0)).size()
    return image.setMulti({'wc':q1,'nwc':q0})

In [ ]:
training = 1000
collect = 500
def season(image):
    samples = image.updateMask(seasonWaterExtent_min).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': training,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    label = 'waterclass'
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
    cluster_samples = ee.FeatureCollection('users/311605001111/daqing_new_noseason2020')
    clusterer = ee.Clusterer.wekaXMeans(2,2).train(cluster_samples,bands)
    q = ee.FeatureCollection(samples.cluster(clusterer))
    q1 = q.filter(ee.Filter.eq('cluster',1))
    q0 = q.filter(ee.Filter.eq('cluster',0))
    qq1 = ee.Algorithms.If(q1.aggregate_mean('AWEI').gt(0),q1.map(lambda i:i.set({'waterclass':1})),q1.map(lambda i:i.set({'waterclass':0})))
    qq0 = ee.Algorithms.If(q0.aggregate_mean('AWEI').gt(0),q0.map(lambda i:i.set({'waterclass':1})),q0.map(lambda i:i.set({'waterclass':0})))
    cluster_result = ee.FeatureCollection([qq1,qq0]).flatten()
    trainedClassifier = ee.Classifier.smileRandomForest(20).train(cluster_result,label,bands)
    classify_image = image.updateMask(seasonWaterExtent_max).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')   
    seasonWater = image.updateMask(classify_image).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': collect,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    sw = seasonWater.map(lambda i : i.setMulti({'waterclass':1,'Image_id':image.get('system:id')}))
    seasonLand = image.updateMask(classify_image.remap([0,1],[1,0])).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': collect,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    sl = seasonLand.map(lambda i : i.setMulti({'waterclass':0,'Image_id':image.get('system:id')}))
    sample_points = ee.FeatureCollection([sw,sl]).flatten()
    return sample_points

## 季节性，k-mean

In [ ]:
training = 500
# collect = 500

In [ ]:
def filter_image(image):
    samples = image.updateMask(seasonWaterExtent_min).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': training,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    label = 'waterclass'
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
    cluster_samples = ee.FeatureCollection('users/311605001111/tibet_kmean100_noseason_sample2016')
    clusterer = ee.Clusterer.wekaKMeans(2).train(cluster_samples,bands)
    q = ee.FeatureCollection(samples.cluster(clusterer))
    q1 = q.filter(ee.Filter.eq('cluster',1)).size()
    q0 = q.filter(ee.Filter.eq('cluster',0)).size()
    return image.setMulti({'wc':q1,'nwc':q0})

In [ ]:
# def season(image):
#     samples = image.updateMask(seasonWaterExtent_min).sample(**{
#         'region': roi,
#         'scale': 30,
#         'numPixels': training,
#         'seed': 0,
#         'geometries': True,
#         'tileScale': 2,
#     })
#     season_points = samples.map(lambda i : i.setMulti({'point_type':'season_water','Image_id':image.get('system:id')}))
#     label = 'waterclass'
#     bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
#     cluster_samples = ee.FeatureCollection('users/311605001111/tianjin_kmean100_noseason_sample2020')
#     clusterer = ee.Clusterer.wekaKMeans(2).train(cluster_samples,bands)
#     q = ee.FeatureCollection(season_points.cluster(clusterer))
#     q1 = q.filter(ee.Filter.eq('cluster',1))
#     q0 = q.filter(ee.Filter.eq('cluster',0))
#     qq1 = ee.Algorithms.If(q1.aggregate_mean('AWEI').gt(0),q1.map(lambda i:i.set({'waterclass':1})),q1.map(lambda i:i.set({'waterclass':0})))
#     qq0 = ee.Algorithms.If(q0.aggregate_mean('AWEI').gt(0),q0.map(lambda i:i.set({'waterclass':1})),q0.map(lambda i:i.set({'waterclass':0})))
#     cluster_result = ee.FeatureCollection([qq1,qq0]).flatten()
#     return cluster_result

In [ ]:
def season(image):
    samples = image.updateMask(seasonWaterExtent_min).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': training,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    season_points = samples.map(lambda i : i.setMulti({'point_type':'season_water','Image_id':image.get('system:id')}))
    label = 'waterclass'
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
    cluster_samples = ee.FeatureCollection('users/311605001111/tibet_kmean100_noseason_sample2016')
    clusterer = ee.Clusterer.wekaKMeans(2).train(cluster_samples,bands)
    q = ee.FeatureCollection(season_points.cluster(clusterer))
    q1 = q.filter(ee.Filter.eq('cluster',1))
    q0 = q.filter(ee.Filter.eq('cluster',0))
    b1 = ee.FeatureCollection(q1).filter(ee.Filter.lt('AWEI',0)).size()
    b2 = ee.FeatureCollection(q1).filter(ee.Filter.gt('AWEI',0)).size()
    c1 = ee.FeatureCollection(q0).filter(ee.Filter.lt('AWEI',0)).size()
    c2 = ee.FeatureCollection(q0).filter(ee.Filter.gt('AWEI',0)).size()
    qq1 = ee.Algorithms.If(ee.Number(b1).lt(ee.Number(b2)),q1.map(lambda i:i.set({'waterclass':1})),q1.map(lambda i:i.set({'waterclass':0})))
    qq0 = ee.Algorithms.If(ee.Number(c1).lt(ee.Number(c2)),q0.map(lambda i:i.set({'waterclass':1})),q0.map(lambda i:i.set({'waterclass':0})))
    cluster_result = ee.FeatureCollection([qq1,qq0]).flatten()
    return cluster_result


## 季节性，随机选点

In [11]:
## 设置采样个数
training = 1500

def season(image):
    season_points = image.updateMask(seasonWaterExtent_min).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': training,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    water_points = season_points.map(lambda i : i.setMulti({'point_type':'season','Image_id':image.get('system:id')}))
    return water_points

# 抽样

In [ ]:
## 永久性水体样本和非水体样本，抽样
pw_nw = sample_image.map(imageSample).flatten()
permanent_water = pw_nw.filter(ee.Filter.eq('waterclass',1)).randomColumn('random',1,'uniform').filter(ee.Filter.lt('random',0.2))
no_water = pw_nw.filter(ee.Filter.eq('waterclass',0)).randomColumn('random',1,'uniform').filter(ee.Filter.lt('random',0.2))

## 季节性样本，抽样
sw = sample_image.map(season).flatten().randomColumn('random',1,'uniform').filter(ee.Filter.lt('random',0.4))

## 季节性样本，分类
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
cluster_samples = ee.FeatureCollection([permanent_water,no_water]).flatten()
clusterer = ee.Clusterer.wekaKMeans(2).train(cluster_samples,bands)
q = ee.FeatureCollection(sw.cluster(clusterer))
q1 = q.filter(ee.Filter.eq('cluster',1))
q0 = q.filter(ee.Filter.eq('cluster',0))
b1 = ee.FeatureCollection(q1).filter(ee.Filter.lt('AWEI',0)).size()
b2 = ee.FeatureCollection(q1).filter(ee.Filter.gt('AWEI',0)).size()
c1 = ee.FeatureCollection(q0).filter(ee.Filter.lt('AWEI',0)).size()
c2 = ee.FeatureCollection(q0).filter(ee.Filter.gt('AWEI',0)).size()
qq1 = ee.Algorithms.If(ee.Number(b1).lt(ee.Number(b2)),q1.map(lambda i:i.set({'waterclass':1})),q1.map(lambda i:i.set({'waterclass':0})))
qq0 = ee.Algorithms.If(ee.Number(c1).lt(ee.Number(c2)),q0.map(lambda i:i.set({'waterclass':1})),q0.map(lambda i:i.set({'waterclass':0})))
seasonWater = ee.FeatureCollection([qq1,qq0]).flatten()

## 合并
total_Samples = ee.FeatureCollection([permanent_water,no_water,seasonWater]).flatten()

## 异常值去除
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
#### cluster与waterclass的对比
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft
#### k-mean聚类
def k_mean(sampleSET):
    clusterer = ee.Clusterer.wekaKMeans(2).train(sampleSET,bands)
    result = sampleSET.cluster(clusterer)
    right = result.map(Compare).filter(ee.Filter.eq('eq',0))
    error = result.map(Compare).filter(ee.Filter.eq('eq',1))
    filtered_sample = ee.FeatureCollection(ee.Algorithms.If(right.size().gt(error.size()),right,error))
    return filtered_sample

filtered_samples = k_mean(total_Samples)

In [ ]:
# 总的样本点数
points_collection = k_mean(total_Samples)
dataset_id = region + '_sample_' + year
assetID = 'users/311605001111/' + dataset_id
task = ee.batch.Export.table.toAsset(**{
    'collection': points_collection,
    'description': dataset_id,
    'assetId': assetID
})
task.start()

# 基于影像，进行样本采集

In [12]:
## 永久性水体样本和非水体样本，抽样
# pw_nw = sample_image.map(imageSample).flatten()
# permanent_water = pw_nw.filter(ee.Filter.eq('waterclass',1)).randomColumn('random',1,'uniform').filter(ee.Filter.lt('random',0.2))
# no_water = pw_nw.filter(ee.Filter.eq('waterclass',0)).randomColumn('random',1,'uniform').filter(ee.Filter.lt('random',0.2))
# points_collection = ee.FeatureCollection([permanent_water,no_water]).flatten()
# 总的样本点数
points_collection = sample_image.map(imageSample).flatten()
dataset_id = region + '_kmean100_noseason_v2' + year
assetID = 'users/311605001111/' + dataset_id
task = ee.batch.Export.table.toAsset(**{
    'collection': points_collection,
    'description': dataset_id,
    'assetId': assetID
})
task.start()

In [13]:
# 随机采样，并输出
# q3 = sample_image.map(season,True).flatten()
# label = 'waterclass'
# bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
# cluster_samples = ee.FeatureCollection('users/311605001111/wuhan_kmean85_noseason2020')
# clusterer = ee.Clusterer.wekaKMeans(2).train(cluster_samples,bands)
# q = ee.FeatureCollection(q3.cluster(clusterer))
# q1 = q.filter(ee.Filter.eq('cluster',1))
# q0 = q.filter(ee.Filter.eq('cluster',0))
# qq1 = ee.Algorithms.If(q1.aggregate_mean('AWEI').gt(0),q1.map(lambda i:i.set({'waterclass':1})),q1.map(lambda i:i.set({'waterclass':0})))
# qq0 = ee.Algorithms.If(q0.aggregate_mean('AWEI').gt(0),q0.map(lambda i:i.set({'waterclass':1})),q0.map(lambda i:i.set({'waterclass':0})))
# points_collection = ee.FeatureCollection([qq1,qq0]).flatten()

points_collection = sample_image.map(season).flatten()
dataset_id = region + '_kmean100_season_v2' + year
assetID = 'users/311605001111/' + dataset_id
task = ee.batch.Export.table.toAsset(**{
    'collection': points_collection,
    'description': dataset_id,
    'assetId': assetID
})
task.start()

In [ ]:
# points_collection = sample_image.map(imageSample).flatten()
points_collection = sample_image.map(filter_image).filter(ee.Filter.gt('wc',0)).filter(ee.Filter.gt('nwc',0)).map(season).flatten()

dataset_id = region + '_kmean100_season_sample' + year
assetID = 'users/311605001111/' + dataset_id
task = ee.batch.Export.table.toAsset(**{
    'collection': points_collection,
    'description': dataset_id,
    'assetId': assetID
})
task.start()

In [ ]:
folder = 'wuhan'
label = 'wuhan_new' + year + '_l78_awei0'
# label = 'tianjin_' + 'new2020_l78'
# csv格式
selector = ['Image_id','B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI','waterclass']
geemap.ee_export_vector_to_drive(points_collection, description=label, folder=folder, file_format='csv', selectors=selector)

# shp格式
selector = ['Image_id','B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI','waterclass']
geemap.ee_export_vector_to_drive(points_collection, description=label, folder=folder, file_format='shp', selectors=selector)

In [ ]:
image = maskSR(water_index(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_119028_20200316'))).clip(roi)
visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}
Map.addLayer(image.clip(roi), visParams,'image')

In [ ]:
#### 设置年份
JRC_id = 'JRC/GSW1_3/YearlyHistory/' + year
Maryland_id = 'users/311605001111/Maryland/Maryland_nationwide_' + year
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
# JRC
JRC = ee.Image(JRC_id).clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
JRC_waterclass = ee.ImageCollection([JRC,basemap]).sum()
# Maryland
Maryland = ee.Image(Maryland_id).clip(roi).select('b1').rename('waterclass')
Maryland_permanent = Maryland.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
Maryland_season= ee.ImageCollection([Maryland.gte(25),Maryland.lt(75)]).sum().eq(2)
Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()
# 三者的交集
permanentWaterExtent = ee.ImageCollection([JRC_waterclass.eq(2),Maryland_waterclass.eq(2)]).sum().eq(2)
landExtent = ee.ImageCollection([JRC_waterclass.eq(0),Maryland_waterclass.eq(0)]).sum().eq(2)
seasonWaterExtent_min = ee.ImageCollection([JRC_waterclass.eq(1),Maryland_waterclass.eq(1)]).sum().eq(2)
seasonWaterExtent_max = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')

In [ ]:
def filter_image(image):
    samples = image.updateMask(seasonWaterExtent_min).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': training,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    label = 'waterclass'
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
    cluster_samples = ee.FeatureCollection('users/311605001111/daqing_new_noseason2020')
    clusterer = ee.Clusterer.wekaXMeans(2,2).train(cluster_samples,bands)
    q = ee.FeatureCollection(samples.cluster(clusterer))
    q1 = q.filter(ee.Filter.eq('cluster',1)).size()
    q0 = q.filter(ee.Filter.eq('cluster',0)).size()
    return image.setMulti({'wc':q1,'nwc':q0})

In [ ]:
col_samples = image.updateMask(seasonWaterExtent_min).sample(**{
    'region': roi,
    'scale': 30,
    'numPixels': 2000,
    'seed': 0,
    'geometries': True,
    'tileScale': 2,
})

bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
samples = ee.FeatureCollection('users/311605001111/daqing_new_noseason2020')
clusterer = ee.Clusterer.wekaKMeans(2).train(samples,bands)
q = ee.FeatureCollection(col_samples.cluster(clusterer))
q1 = q.filter(ee.Filter.eq('cluster',1))
q0 = q.filter(ee.Filter.eq('cluster',0))
qq1 = ee.Algorithms.If(q1.aggregate_mean('AWEI').gt(0),q1.map(lambda i:i.set({'waterclass':1})),q1.map(lambda i:i.set({'waterclass':0})))
qq2 = ee.Algorithms.If(q0.aggregate_mean('AWEI').gt(0),q0.map(lambda i:i.set({'waterclass':1})),q0.map(lambda i:i.set({'waterclass':0})))
qq = ee.FeatureCollection([qq1,qq2]).flatten()
print(q.size().getInfo())
print(q.first().getInfo())
print(q.aggregate_array('cluster').getInfo())
print(q.aggregate_array('waterclass').getInfo())

print(q1.aggregate_mean('AWEI').getInfo())
print(q0.aggregate_mean('AWEI').getInfo())

In [ ]:
id = ['LANDSAT/LE07/C01/T1_SR/LE07_122039_20150212', 'LANDSAT/LE07/C01/T1_SR/LE07_123039_20150307', 'LANDSAT/LE07/C01/T1_SR/LE07_123039_20150611', 'LANDSAT/LE07/C01/T1_SR/LE07_123039_20150713', 'LANDSAT/LE07/C01/T1_SR/LE07_123039_20150729', 'LANDSAT/LE07/C01/T1_SR/LE07_123039_20150830', 'LANDSAT/LE07/C01/T1_SR/LE07_123039_20150915', 'LANDSAT/LE07/C01/T1_SR/LE07_123039_20151017', 'LANDSAT/LE07/C01/T1_SR/LE07_123039_20151102']

for i in id:
    print(i)
    image = maskSR(water_index(ee.Image(ee.String(i)).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']))).clip(roi)
    col_samples = image.updateMask(seasonWaterExtent_min).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': 2000,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
    samples = ee.FeatureCollection('users/311605001111/wuhan_new_noseason2020')
    clusterer = ee.Clusterer.wekaKMeans(2).train(samples,bands)
    q = ee.FeatureCollection(col_samples.cluster(clusterer))
    q1 = q.filter(ee.Filter.eq('cluster',1))
    q0 = q.filter(ee.Filter.eq('cluster',0))
    print(q1.aggregate_mean('AWEI').getInfo())
    print(q0.aggregate_mean('AWEI').getInfo())  